In [1]:
import numpy as np
import mujoco as mujoco
import matplotlib.pyplot as plt
import matplotlib.animation as animate

import mujoco_funcs as mj_funcs
import mjcf_models as mj_models
import visualize_mj_funcs as mj_vis

import dyn_functions as dyn
import visualize_dyn_funcs as vis_dyn
import gen_ctrl_funcs as gen_ctrl
import analyze_ilqr_output_funcs as analyze
np.set_printoptions(precision=5, suppress=True, linewidth=100)


In [2]:
dt = 0.05
len_seq = 200
time_vec = np.arange(0,len_seq*dt, dt)
h_bar = 1.0
r_bar = 0.05
m_bar = 1.0
moi = (1/12)*h_bar*(m_bar**2 + 3*r_bar**2)
x_init = np.array([0.0,0.0,0.0, 0.0])
u_vec = np.array([0.0]) 
x_seq = np.zeros((len_seq, 4))
x_seq[0]= x_init

In [3]:
mjcf_model  = mj_models.create_MJCF_double_pend_dev()
mj_model, mj_render, mj_data = mj_funcs.create_mujoco_model(mjcf_model, dt)

In [4]:

nu = mj_model.nu
nx = mj_model.nv    
eps = 1e-6
A = np.zeros((2*nx, 2*nx))
B = np.zeros((2*nx,   nu))
flg_centered = False
mj_model.opt.timestep = dt
mj_data.ctrl = u_vec
mujoco.mj_forward(mj_model,mj_data)
print(mj_data.qpos)
print(mj_data.qvel)
print(mj_data.ctrl)

[0. 0.]
[0. 0.]
[0.]


In [5]:
dpend_sys = dyn.double_pend_rel_dyn(g=9.81, m1=m_bar, moi1=moi, d1=h_bar/2, l1=h_bar, 
                                            m2=m_bar, moi2=moi, d2=h_bar/2, l2=h_bar, 
                                        b1=0.0, b2=0.0, shoulder_act=True, elbow_act=False)
disc_dyn_func = lambda x, u: gen_ctrl.step_rk4(dpend_sys.cont_dyn_func, dt, x, u)


In [6]:
mujoco.mj_forward(mj_model, mj_data)
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps, flg_centered)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, x_init, u_vec)

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.96852  0.03135  0.05     0.     ]
 [ 0.04189  0.885    0.       0.05   ]
 [-0.62957  0.62706  1.       0.     ]
 [ 0.83775 -2.30005  0.       1.     ]]
A dyn: 
 [[ 0.98436  0.01549  0.04974  0.00026]
 [ 0.02069  0.94309  0.00035  0.04904]
 [-0.62195  0.61196  0.98436  0.01549]
 [ 0.81748 -2.25232  0.02069  0.94309]]
B mujoco: 
 [[ 0.00427]
 [-0.01066]
 [ 0.0854 ]
 [-0.21324]]
B dyn: 
 [[ 0.00212]
 [-0.00527]
 [ 0.08385]
 [-0.20862]]


In [19]:
x_vec = np.array([1.0, 1.0, 0.0, 0.0])
u_vec = np.array([0.0,0.0])
mj_funcs.set_mj_ctrl_vec(mj_data, u_vec)
mj_funcs.set_mj_state_vec(mj_data, x_vec)
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps=1e-6, flg_centered=True)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, x_vec, u_vec)

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.97848 -0.00429  0.05     0.     ]
 [ 0.0542   0.99827  0.       0.05   ]
 [-0.43037 -0.08586  1.       0.     ]
 [ 1.08404 -0.03458  0.       1.     ]]
A dyn: 
 [[ 0.98922 -0.00216  0.04961 -0.00015]
 [ 0.02699  0.99913  0.00124  0.05019]
 [-0.43203 -0.08659  0.97648 -0.00885]
 [ 1.07508 -0.0353   0.07442  1.0111 ]]
B mujoco: 
 [[ 0.00224 -0.00405]
 [-0.00405  0.01482]
 [ 0.04482 -0.08108]
 [-0.08108  0.29636]]
B dyn: 
 [[ 0.00112 -0.00203]
 [-0.00201  0.0074 ]
 [ 0.0447  -0.08161]
 [-0.07986  0.2953 ]]


In [ ]:
pot_energy_seq = np.zeros((len_seq,1))
kin_energy_seq = np.zeros((len_seq,1))
tot_energy_seq = np.zeros((len_seq,1))
pot_energy_seq[0] = dpend_sys.calculate_potential_energy(x_init)
kin_energy_seq[0] = dpend_sys.calculate_kinetic_energy(x_init)
tot_energy_seq[0] = dpend_sys.calculate_total_energy(x_init)